In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()


,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Khatanga,RU,2020-10-04 01:30:30,71.97,102.50,24.28,98,100,13.13
1,1,Thompson,CA,2020-10-04 01:30:30,55.74,-97.86,39.20,80,75,3.36
2,2,Nome,US,2020-10-04 01:30:30,64.50,-165.41,48.99,87,90,3.33
3,3,Butaritari,KI,2020-10-04 01:30:30,3.07,172.79,81.37,71,100,8.72
4,4,Busselton,AU,2020-10-04 01:30:30,-33.65,115.33,62.01,50,4,11.79


In [3]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [6]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False, max_intensity=300, point_radius=5)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=5)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=5)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [10]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
3,3,Butaritari,KI,2020-10-04 01:30:30,3.07,172.79,81.37,71,100,8.72
7,7,Kapaa,US,2020-10-04 01:30:31,22.08,-159.32,87.80,66,1,9.17
13,13,Lamu,KE,2020-10-04 01:30:31,-2.27,40.90,77.00,83,20,4.70
16,16,Dingle,PH,2020-10-04 01:30:32,11.00,122.67,86.27,70,37,1.77
20,20,Humaita,BR,2020-10-04 01:30:32,-7.51,-63.02,76.60,82,72,2.59
25,25,Sur,OM,2020-10-04 01:30:33,22.57,59.53,81.59,45,0,2.77
28,28,Neiafu,TO,2020-10-04 01:30:33,-18.65,-173.98,78.80,88,75,13.87
35,35,Hilo,US,2020-10-04 01:30:34,19.73,-155.09,86.00,58,1,12.75
38,38,Atuona,PF,2020-10-04 01:30:34,-9.80,-139.03,78.46,73,0,20.13
39,39,Bambanglipuro,ID,2020-10-04 01:30:34,-7.95,110.28,82.22,70,99,14.27


In [11]:
preferred_cities_df.count()

City_ID       184
City          184
Country       184
Date          184
Lat           184
Lng           184
Max Temp      184
Humidity      184
Cloudiness    184
Wind Speed    184
dtype: int64

In [12]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
3,Butaritari,KI,81.37,3.07,172.79,
7,Kapaa,US,87.80,22.08,-159.32,
13,Lamu,KE,77.00,-2.27,40.90,
16,Dingle,PH,86.27,11.00,122.67,
20,Humaita,BR,76.60,-7.51,-63.02,
25,Sur,OM,81.59,22.57,59.53,
28,Neiafu,TO,78.80,-18.65,-173.98,
35,Hilo,US,86.00,19.73,-155.09,
38,Atuona,PF,78.46,-9.80,-139.03,
39,Bambanglipuro,ID,82.22,-7.95,110.28,


In [18]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
         hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
         print("Hotel not found... skipping.")
    

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [20]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
3,Butaritari,KI,81.37,3.07,172.79,Isles Sunset Lodge
7,Kapaa,US,87.80,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
13,Lamu,KE,77.00,-2.27,40.90,Lamu House
16,Dingle,PH,86.27,11.00,122.67,Camp Pasica
20,Humaita,BR,76.60,-7.51,-63.02,Corrêa Hotel
25,Sur,OM,81.59,22.57,59.53,Sur Plaza Hotel
28,Neiafu,TO,78.80,-18.65,-173.98,Mystic Sands
35,Hilo,US,86.00,19.73,-155.09,Hilo Hawaiian Hotel
38,Atuona,PF,78.46,-9.80,-139.03,Villa Enata
39,Bambanglipuro,ID,82.22,-7.95,110.28,Puri Brata


In [21]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=5)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [22]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [23]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [24]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))